# Atlas Translation: Yeo-2011 to Desikan-Killiani (DK)

In [ ]:
import itertools
import nibabel as nb
import numpy as np
import pandas as pd
from tqdm import tqdm
from atlasreader import atlasreader as ar
from nilearn._utils import check_niimg

## 1.1 Calculate Voxel Data

In [ ]:
### 1.1 Storing Yeo Atlas information

# # For Yeo 7 networks:
# #Read Yeo labels from file, fixing spacings and new / empty lines
# Yeo_labels = open('../data/atlases/Yeo2011_7Networks_MNI152.txt', 'r')
# Yeo_labels = Yeo_labels.read().split('\n')
# #Yeo_labels = [Yeo_labels[i][8:] for i in range(len(Yeo_labels)-1)]
# Yeo_labels.insert(0,"None") #I believe the areas were numbered 1-7 with "0" reserved for white matter

# #Read Yeo matrix
# Yeo_template = "../data/atlases/Yeo2011_7Networks_MNI152.nii"
# Yeo = nb.load(Yeo_template)
# Yeo_data = Yeo.get_data()

# For Yeo 17 Networks:
# Read Yeo labels from file, fixing spacing and new/empty lines
Yeo_labels = open('../data/atlases/Yeo/Yeo2011_17Networks_MNI152.txt', 'r')
Yeo_labels = Yeo_labels.read().split('\n')
#Yeo_labels = [Yeo_labels[i][17:] for i in range(len(Yeo_labels)-1)]
Yeo_labels.insert(0,"None") #I believe the areas were numbered 1-7 with "0" reserved for white matter

#Read Yeo matrix
Yeo_template = "../data/atlases/Yeo/Yeo2011_17Networks_MNI152.nii"
Yeo = nb.load(Yeo_template)
Yeo_data = Yeo.get_data()

### 1.2 Loop thru MNI-152 template data

In [ ]:
DK_data = ar.get_atlas("desikan_killiany")['image'].get_data()

DK_atlas = ar.get_atlas("desikan_killiany")

We would like to loop over the pixels and compare them between atlases but, unfortunately, the matrices are cropped to remove large regions full of zeros so the matrices are not 256x256x256 anymore.

We can try doing the same operation in real, xyz space instead of in MNI-type indices. We do so by using `coord_ijk_to_xyz` function in `atlasreader`.

In [ ]:
Yeo_affine = check_niimg(Yeo_template).affine
DK_affine  = check_niimg(ar.get_atlas("desikan_killiany")['image']).affine

### 1.3. Compute the DK voxel-based distribution of networks per region (very slow!!)

In [ ]:
DK_dict_voxels = {}
DK_dict_counts = {}

for n in tqdm(ar.get_atlas("desikan_killiany")['labels']['name']):
    DK_dict_counts.update({n: {Yeo_label:0 for Yeo_label in Yeo_labels}})
    DK_dict_voxels.update({n: {Yeo_label:[] for Yeo_label in Yeo_labels}})

   
for DK_i in range(0, DK_data.shape[0]):
    for DK_j in range(0, DK_data.shape[1]):
        for DK_k in range(0, DK_data.shape[2]):           
            DK_voxel_label  = DK_data[DK_i, DK_j, DK_k]
            DK_voxel_region = ar.get_label(DK_atlas, DK_voxel_label)

            if DK_voxel_region != 'Unknown':

                xyz = ar.coord_ijk_to_xyz(DK_affine, [DK_i, DK_j, DK_k])
                Yeo_ijk = ar.coord_xyz_to_ijk(Yeo_affine, xyz)[0]
                             
                Yeo_voxel_label = int(Yeo_data[Yeo_ijk[0], Yeo_ijk[1], Yeo_ijk[2]])
                Yeo_voxel_region = Yeo_labels[Yeo_voxel_label]

                DK_dict_counts[DK_voxel_region][Yeo_voxel_region] += 1
                DK_dict_voxels[DK_voxel_region][Yeo_voxel_region].append(xyz[0])

In [ ]:
from scipy.io import savemat

# np.save('DK_dict_voxels.npy', DK_dict_voxels)
# np.save('DK_dict_counts.npy', DK_dict_counts)

# Save 17 networks map... in .mat format?
savemat('DK_dict17_voxels.mat', DK_dict_voxels)
savemat('DK_dict17_counts.mat', DK_dict_counts)

### 1.3b Or just load it

`DK_dict_counts` is a matrix with the number of voxels belonging to a given Yeo network, for each DK region.

`DK_dict_voxels` is a similar matrix but it contains the actual [x,y,z] locations of each voxel so we can use them to make the brain plots

In [ ]:
DK_dict_counts = np.load("DK_dict_counts.npy")
DK_dict_counts = DK_dict_counts.item()

In [ ]:
pd.DataFrame(DK_dict_counts)

## 2. Prepare the data

### 2.1 Make Pandas DataFrame with the 86 ROIs

The DK dataframe from `nilearn` has extra regions like "Left-Cerebellum-White-Matter" that need to be removed. 

I do so below by making a new dataframe containing only columns that agree with what we know to be DK.

In [ ]:
DK_region_names = pd.read_csv("../data/dk_names.csv").set_index('Atlas')

In [ ]:
DK_dict_86_regions = {}
for region in DK_region_names.index:
    DK_dict_86_regions.update({region:DK_dict_counts[region]})

In [ ]:
# transform into a pandas df
DK_dict_counts = pd.DataFrame(DK_dict_86_regions)
# drop "None" (white matter)
DK_dict_counts = DK_dict_counts.drop(['None'])

In [ ]:
DK_dict_counts

### 2.2 Normalize each region so that its networks add up to 1

In [ ]:
DK_df_normalized = DK_dict_counts.div(DK_dict_counts.sum(axis=0), axis=1)

In [ ]:
DK_df_normalized

### 2.3 Save to file

In [ ]:
DK_df_normalized.to_csv("DK_dictionary_normalized.csv")

## 3. Plotting networks

### 3.1 Plot ROI (center of mass only)

In [ ]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
from nilearn import plotting

In [ ]:
# Calculate center of mass from file
com_dk = np.load("../data/com_dk.npy")
com_dk = com_dk.ravel()[0] #not sure why / whether i need this
coords = np.array([com_dk[region] for region in DK_df_normalized.columns])

In [ ]:
def plot_glass_brains(color, coords, size):
    """Plot a glass brain for a 90 regions ATLAS with nodes colored by `color`.

    Args:
        color (list): Color indices. e.g. [0,1,1,0] will color nodes [1,2] differently.

    Returns:
        matplotlib.plot: A plot object.

    """

    connec = np.array([[0]*86]*86)

    plotting.plot_connectome(connec, coords, node_size = size, node_color=color, display_mode='lyrz')

In [ ]:
plot_glass_brains([[0.9921875,0.64453125,1]]*86, coords, DK_df_normalized.loc['Visual'].values*120)

In [ ]:
plot_glass_brains([[0.9921875,0.2,0.64453125]]*86, coords, DK_df_normalized.loc['Limbic'].values*120)

In [ ]:
plot_glass_brains([[0.1,0.64453125,0.64453125]]*86, coords, DK_df_normalized.loc['Default'].values*120)

In [ ]:
plot_glass_brains([[0.1,0.1,0.64453125]]*86, coords, DK_df_normalized.loc['Somatomotor'].values*120)

In [ ]:
plot_glass_brains([[0.5,0.3,0.1]]*86, coords, DK_df_normalized.loc['Frontoparietal'].values*120)

In [ ]:
plot_glass_brains([[0.6,0.6,0.8]]*86, coords, DK_df_normalized.loc['Ventral_Attention'].values*120)

In [ ]:
plot_glass_brains([[0.9921875,0.64453125,0.1]]*86, coords, DK_df_normalized.loc['Dorsal_Attention'].values*220)

### 3.2 Plotting all voxels

In [ ]:
DK_dict_voxels = np.load("DK_dict_voxels.npy")

In [ ]:
DK_dict_voxels = DK_dict_voxels.item()

In [ ]:
DK_voxels_df = pd.DataFrame(DK_dict_voxels)

In [ ]:
network_coordinates_DK = {network:[] for network in DK_voxels_df.index}

In [ ]:
colors = np.array(
         [[255,227,133,0.1*255],
          [197,232,146,0.1*255],
          [255,196,158,0.1*255],
          [200,180,232,0.1*255],
          [141,216,255,0.1*255],
          [255,158,179,0.1*255],
          [143,255,200,0.1*255]])/255

In [ ]:
for network in network_coordinates_DK:
    coords = DK_voxels_df.loc[network]
    network_coordinates_DK[network] = np.array(list(itertools.chain.from_iterable(coords)))

In [ ]:
coords = network_coordinates_DK['Visual'][1::10]
coords_len = len(network_coordinates_DK['Visual'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[0]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Default'][1::10]
coords_len = len(network_coordinates_DK['Default'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[1]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Somatomotor'][1::10]
coords_len = len(network_coordinates_DK['Somatomotor'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[2]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Dorsal_Attention'][1::10]
coords_len = len(network_coordinates_DK['Dorsal_Attention'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[3]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Frontoparietal'][1::10]
coords_len = len(network_coordinates_DK['Frontoparietal'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[4]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Limbic'][1::10]
coords_len = len(network_coordinates_DK['Limbic'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[5]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
coords = network_coordinates_DK['Ventral_Attention'][1::10]
coords_len = len(network_coordinates_DK['Ventral_Attention'][1::10])
plotting.plot_connectome(np.array([[0]*coords_len]*coords_len),  
                         coords,
                         node_size = 10, 
                         node_color = [colors[6]]*coords_len, 
                         display_mode='lyrz')

In [ ]:
network_coordinates_DK.keys()